In [9]:
import spotipy
import spotipy.util
from spotipy.oauth2 import SpotifyClientCredentials
import os, sys
from pprint import pprint
from spotify_functions import get_track_features, add_track_to_playlist
import yaml
import json
import itertools

In [10]:
# Log in
client_id = "eb61abc5e043451ead922883d9ffe41a"
client_secret = "8e1e6cd54c0948a98702cb977b51e802"
redirect_uri = "http://google.com/"
username = "andersmb"
scope = ["user-library-read",
         "playlist-read-private", 
         "playlist-modify-public",
         "playlist-modify-private"]

scope = " ".join(scope)
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = spotipy.util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token) 



In [3]:
# Get the search keywords
keywords = ["workout",
            "running", 
            "interval training", 
            "jogging", 
            "crossfit", 
            "satselixia", 
            "spinning",
            "high intensity interval training",
            "interval training",
            "hiit",
            "trance",
            "90s",
            "90s music",
            "90s hits",
            "2000 hits",
            "2000s",
            "gym music",
            "party music",
            "party playlist",
            "epic party",
            "top hits",
            "one hit wonders",
            "epic rock riffs",
            "top guitar riffs",
            "metal",
            "hard metal",
            "classic metal",
            "heavy metal",
            "training",
            "training motivation",
            "workout motivation",
            "fittness 24 seveven",
            "fitness24seven",
            "new hits",
            "top hits",
            "2018 hits",
            "2017 hits",
            "2016 hits",
            "2015 hits",
            "2014 hits",
            "2013 hits",
            "top pop hits"]

In [14]:
candidates = {}
temporange1 = list(range(145, 155))
temporange2 = list(range(min(temporange1)/2, max(temporange1)/2))
playlist_search_limit = 3

for i, key in list(enumerate(keywords)):
    sys.stdout.write("Searching playlists {}/{}\r".format(i+1, len(keywords)))
    sys.stdout.flush()
    
    query = sp.search(q=key, limit=playlist_search_limit, type="playlist")
    for plist in query["playlists"]["items"]:
        plist_id = plist["id"]
        owner_id = plist["owner"]["id"]
        data = sp.user_playlist(owner_id, plist_id)
        for track in data["tracks"]["items"]:
            tid = str(track["track"]["id"])
            try:
                tempo = get_track_features(token=token, track_id=tid)["tempo"]
            except:
                continue
            if int(tempo) in temporange1 or int(tempo) in temporange2:
                candidates[tid] = get_track_features(token=token, track_id=tid)
                candidates[tid]["preview_url"] = track["track"]["preview_url"]
                candidates[tid]["artist"] = track["track"]["artists"][0]["name"]
                candidates[tid]["name"] = track["track"]["name"]
                
with open("candidates_anders.json", "w") as f:
    json.dump(candidates, f, indent=4)

In [12]:
# Post to the playlist
# Perhaps filter based on energy?
with open("candidates_eirik.json") as f:
    data = json.load(f)
    
plid_anders = "7tQtcCKxoXCIp8gHZEzK0F"
plid_eirik = "0ggUeDXX7r9Wbpqwksk4fw"
for track in data.keys():
    sp.user_playlist_add_tracks(username, plid_eirik, [track])